<a href="https://colab.research.google.com/github/kimdonggyu2008/2024_2_Capstone/blob/main/translator_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00


#transformer huggingface에서 로드하는 경우

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("KETI-AIR/long-ke-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("KETI-AIR/long-ke-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.49k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.17M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/891 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

#로컬에서 로딩하는 경우

In [4]:
import torch
import pandas as pd
from datasets import load_dataset
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

In [5]:
# # 1. 토크나이저 로드
# tokenizer = AutoTokenizer.from_pretrained("path/to/tokenizer")

# # 2. 모델 로드
# model = AutoModelForSeq2SeqLM.from_pretrained("path/to/model", local_files_only=True)

#학습데이터 로드 및 토크나이징

In [6]:

# 3. 데이터셋 로드 (예시 데이터셋)
df = pd.read_csv('/content/drive/MyDrive/translator/data/train.csv')
dataset = Dataset.from_pandas(df)


In [7]:
# 4. 데이터 전처리 함수 (한국어 -> 영어 번역)
def preprocess_function(examples):
    inputs = examples['ko']  # 'ko' 열에서 한국어 텍스트를 가져옴
    targets = examples['en']  # 'en' 열에서 영어 번역 텍스트를 가져옴
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # T5 모델은 입력과 출력이 모두 시퀀스이므로 target도 tokenizing이 필요
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [8]:

# 5. 데이터셋 전처리
tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/833321 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [9]:
df_train = pd.DataFrame({
    'input_ids': tokenized_datasets['input_ids'],
    'attention_mask': tokenized_datasets['attention_mask'],
    'labels': tokenized_datasets['labels']
})


In [10]:

# # 7. 토크나이징된 데이터를 CSV 파일로 저장
# df_train.to_csv('/content/drive/MyDrive/translator/preprocessed/tokenized_train_data.csv', index=False)
# print("토크나이징된 train 데이터를 CSV로 저장했습니다.")

In [11]:
import json
df_train.to_json('/content/drive/MyDrive/translator/preprocessed/tokenized_data.json', orient='records', lines=True)

print("데이터를 JSON 형식으로 저장했습니다.")

데이터를 JSON 형식으로 저장했습니다.


#학습(토크나이징 된 데이터 받음)


In [ ]:
# 토크나이징 된 CSV 파일 불러오기
df_train_loaded = pd.read_csv('/content/drive/MyDrive/translator/preprocessed/tokenized_train_data.csv')


In [ ]:

# 불러온 데이터를 리스트로 변환
input_ids = df_train_loaded['input_ids'].apply(lambda x: eval(x)).tolist()
attention_mask = df_train_loaded['attention_mask'].apply(lambda x: eval(x)).tolist()
labels = df_train_loaded['labels'].apply(lambda x: eval(x)).tolist()


In [11]:

# PyTorch Tensor로 변환
input_ids = torch.tensor(input_ids)
attention_mask = torch.tensor(attention_mask)
labels = torch.tensor(labels)

# 불러온 데이터를 학습에 사용
train_data = {
    'input_ids': input_ids,
    'attention_mask': attention_mask,
    'labels': labels
}

print("CSV 파일에서 데이터를 불러왔습니다.")

CSV 파일에서 데이터를 불러왔습니다.


In [17]:
# 학습 데이터 일부 출력
print("Input IDs:", input_ids[:2])  # 첫 두 개의 input_ids
print("Attention Mask:", attention_mask[:2])  # 첫 두 개의 attention_mask
print("Labels:", labels[:2])  # 첫 두 개의 labels

Input IDs: tensor([[  782, 16800,  1798,  ...,     0,     0,     0],
        [15863,   387, 20087,  ...,     0,     0,     0]])
Attention Mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
Labels: tensor([[39724, 20462, 21671,  ...,     0,     0,     0],
        [20442, 20007, 20079,  ...,     0,     0,     0]])


In [13]:

# 6. 학습 파라미터 설정 (중간에 멈춰도 저장되도록 설정)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/translator/model",  # 모델 및 체크포인트가 저장될 디렉토리
    evaluation_strategy="steps",  # 매 에포크마다 평가
    save_strategy="steps",  # 스텝마다 체크포인트 저장
    save_steps=500,  # 500 스텝마다 체크포인트 저장
    save_total_limit=3,  # 최근 3개의 체크포인트만 유지
    logging_steps=100,  # 로그를 출력할 스텝 간격
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,  # 학습 종료 시 가장 성능이 좋은 모델 불러오기
    save_on_each_node=False,  # 모든 노드에서 체크포인트를 저장하지 않음 (분산 학습 시)
    logging_dir="/content/drive/MyDrive/translator/logging",  # 로그를 저장할 디렉토리
)


In [14]:

# 7. 트레이너 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
)


In [16]:

# 8. 학습 시작
try:
    trainer.train()
except KeyboardInterrupt:
    print("학습이 중단되었습니다. 마지막 체크포인트가 저장되었습니다.")


KeyError: 0

In [ ]:

# 9. 최종 모델 저장
trainer.save_model("./results/final_model")


In [ ]:

# 10. 토크나이저 저장 (필요할 경우)
tokenizer.save_pretrained("./results/final_model")